In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time
import os

mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils
tips = [4, 8, 12, 16, 20]
brushThickness = 25
eraserThickness = 100

def Track(img, draw=True):
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)

    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            if draw:
                mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)
    return img, results

def Positions(img, results):
    lmList = []
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            for id, lm in enumerate(handLms.landmark):
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                lmList.append([id, cx, cy])
    return lmList

def fingersTop(lmList):
    fingers = []
    if lmList[tips[0]][1] < lmList[tips[0] - 1][1]:
        fingers.append(1)
    else:
        fingers.append(0)

    for id in range(1, 5):
        if lmList[tips[id]][2] < lmList[tips[id] - 2][2]:
            fingers.append(1)
        else:
            fingers.append(0)

    return fingers

def process_header_image(image, target_height=400):
    # Resize image to have a height of 400 pixels while maintaining aspect ratio
    aspect_ratio = image.shape[1] / image.shape[0]
    new_width = int(target_height * aspect_ratio)
    resized_image = cv2.resize(image, (new_width, target_height))
    
    # Create a black background of size 400x200
    background = np.zeros((400, 200, 3), dtype=np.uint8)
    
    # Calculate position to paste the resized image
    y_offset = 0
    x_offset = (200 - new_width) // 2
    
    # Paste the resized image onto the black background
    background[y_offset:y_offset+target_height, x_offset:x_offset+new_width] = resized_image
    
    return background

# Load header images
folderPath = "/Users/saiteja/project/Hand Gesture/assets2"
myList = os.listdir(folderPath)
overlayList = []

if os.path.exists(folderPath):
    myList = os.listdir(folderPath)
    print(f"Found {len(myList)} images in the assets folder")
    for imPath in myList:
        image = cv2.imread(f'{folderPath}/{imPath}')
        if image is not None:
            processed_image = process_header_image(image)
            overlayList.append(processed_image)
        else:
            print(f"Failed to load image: {imPath}")
else:
    print(f"Folder not found: {folderPath}")

if len(overlayList) > 0:
    header = overlayList[0]
    print(f"Header shape: {header.shape}")
else:
    print("No header images loaded. Creating a blank header.")
    header = np.zeros((400, 200, 3), dtype=np.uint8)

drawColor = (255, 0, 255)

cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)

imgCanvas = np.zeros((720, 1280, 3), np.uint8)
xp, yp = 0, 0
pTime = 0

while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)

    img, results = Track(img)
    lmList = Positions(img, results)

    if len(lmList) != 0:
        x1, y1 = lmList[8][1:]
        x2, y2 = lmList[12][1:]

        fingers = fingersTop(lmList)

        if fingers[1] and fingers[2]:
            xp, yp = 0, 0
            print("Selection Mode")
            if x1 < 200:  # Check if within the header area
                if 260 < y1 < 360:
                    header = overlayList[0]
                    drawColor = (255, 0, 255)
                elif 160 < y1 < 260:
                    header = overlayList[1]
                    drawColor = (190, 150, 37)
                elif 360 < y1 < 460:
                    header = overlayList[2]
                    drawColor = (0, 255, 0)
                elif 460 < y1 < 560:
                    header = overlayList[3]
                    drawColor = (0, 0, 0)
            cv2.rectangle(img, (x1, y1 - 25), (x2, y2 + 25), drawColor, cv2.FILLED)

        if fingers[1] and fingers[2] == False:
            cv2.circle(img, (x1, y1), 15, drawColor, cv2.FILLED)
            print("Drawing Mode")
            if xp == 0 and yp == 0:
                xp, yp = x1, y1

            if drawColor == (0, 0, 0):
                cv2.line(img, (xp, yp), (x1, y1), drawColor, eraserThickness)
                cv2.line(imgCanvas, (xp, yp), (x1, y1), drawColor, eraserThickness)
            else:
                cv2.line(img, (xp, yp), (x1, y1), drawColor, brushThickness)
                cv2.line(imgCanvas, (xp, yp), (x1, y1), drawColor, brushThickness)

            xp, yp = x1, y1
        cv2.circle(img, (x1, y1), 10, (0, 255, 255), cv2.FILLED)

    # Combine the canvas with the camera feed
    imgGray = cv2.cvtColor(imgCanvas, cv2.COLOR_BGR2GRAY)
    _, imgInv = cv2.threshold(imgGray, 50, 255, cv2.THRESH_BINARY_INV)
    imgInv = cv2.cvtColor(imgInv, cv2.COLOR_GRAY2BGR)
    img = cv2.bitwise_and(img, imgInv)
    img = cv2.bitwise_or(img, imgCanvas)

    # Add vertical header on the left side
    y_offset = (720 - 400) // 2
    img[y_offset:y_offset+400, 0:200] = header

    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.putText(img, f'FPS: {int(fps)}', (1000, 70), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3)

    cv2.imshow("Image", img)
    cv2.waitKey(1)